## **파일 검색 (File Search)**
- Gemini API를 사용하여 파일 검색 도구를 통해 검색 증강 생성(RAG)를 가능하게 만들어줌
- 제공된 프롬프트를 기반으로 관련 정보를 빠르게 검색할 수 있도록 데이터를 가져오고, 청크로 나누고, 색인을 생성
- 이 정보를 컨텍스트로 사용하여 모델이 더 정확하고 관련성 있는 답변을 할 수 있도록 만듦
- **쿼리**시 **파일 저장** 및 **임베딩 생성**을 무료로 제공
- [Gemini API](https://ai.google.dev/gemini-api/docs/file-search?hl=ko)
- [Google Cookbook](https://github.com/google-gemini/cookbook)

- **장점**
  - 문서를 File Search Store에 업로드하면 Data Loading, Text Splitting, Embedding 및 벡터 스토어 구축이 자동으로 진행
  - 빠르게 RAG를 구성할 수 있음
- **단점**
  - Chunking된 문서에 대한 접근이 어려움
    - 간접적 접근: 검색된 결과를 보는 형태로 가능
  - Chunking 조건을 제외한 다른 부분은 제어할 수 없음

#### **1. 환경 설정**
```
cd {working directory}
uv venv .venv --python 3.13
uv init
uv add google-genai
uv add ipykernel
uv add dotenv
```

In [1]:
from google import genai
from google.genai import types
import time
from dotenv import load_dotenv
import os

In [ ]:

# API Key 불러오기
load_dotenv()

api_key = os.environ.get("GEMINI_API_KEY", None)
if api_key:
    print("GEMINI_API_KEY가 입력되었습니다")
else:
    print("GEMINI_API_KEY가 입력되지 않았습니다.")

GEMINI_API_KEY가 입력되었습니다


#### **1. File Search Store 생성**

In [3]:
client = genai.Client(api_key=api_key)

file_search_store = client.file_search_stores.create(
    config=types.CreateFileSearchStoreConfig(
        display_name='test'
    )
)

print(f"Created store: {file_search_store.name}")

Created store: fileSearchStores/test-es7tw2kwt3b8


#### **2. 파일을 File Search Store에 업로드하기**

In [5]:
upload_op = client.file_search_stores.upload_to_file_search_store(
    file_search_store_name=file_search_store.name,
    file='data/SPRI_AI_Brief_202312.pdf',
    config=types.UploadToFileSearchStoreConfig(
        display_name='SPRI_AI_Brief_2023_12',
    )
)

print(f"Upload started: {upload_op.name}")


while not (upload_op := client.operations.get(upload_op)).done:
    time.sleep(1)
    print(".", end="")

print()
print("Processing complete.")

Upload started: fileSearchStores/test-es7tw2kwt3b8/upload/operations/spriaibrief202312-xq9whafvvfwm

Processing complete.


In [ ]:
# 청크 설정
# operation = client.file_search_stores.upload_to_file_search_store(
#     file_search_store_name=file_search_store.name,
#     file_name=sample_file.name,
#     config={
#         'chunking_config': {
#           'white_space_config': {
#             'max_tokens_per_chunk': 200,
#             'max_overlap_tokens': 20
#           }
#         }
#     }
# )

#### **3. File Search를 활용한 답변**

In [19]:
def query(client, model_id:str, file_store_name:str, contents:str, top_k:int=5):
    response = client.models.generate_content(
            model=model_id,
            contents=contents,
            config=types.GenerateContentConfig(
                tools=[types.Tool(
                    file_search=types.FileSearch(
                        file_search_store_names=[file_store_name],
                        top_k=top_k,
                    )
                )]
            )
        )

    return response

In [20]:
model_id = "gemini-2.5-flash"
file_store_name = file_search_store.name

response = query(client, model_id, file_store_name, contents="이 문서에 대해서 자세히 요약해 주세요.")
print(response.text)

이 문서는 2023년 12월 SPRI 소프트웨어정책연구소에서 발행한 "SPRI AI Brief"로, 인공지능 산업의 최신 동향을 정책/법제, 기업/산업, 기술/연구, 인력/교육 네 가지 측면에서 상세히 다루고 있습니다.

**주요 내용은 다음과 같습니다:**

**1. 정책/법제 동향:**
*   **미국:** 안전하고 신뢰할 수 있는 AI 개발 및 사용에 관한 행정명령을 발표했습니다. 이 명령은 AI의 안전과 보안 기준 마련, 개인정보보호, 형평성과 시민권 향상, 소비자 보호, 노동자 지원, 혁신과 경쟁 촉진, 국제협력을 골자로 합니다. 특히 강력한 AI 시스템을 개발하는 기업에게 안전 테스트 결과 및 주요 정보를 정부와 공유할 것을 요구하고, AI 생성 콘텐츠 표시를 위한 표준 확립을 추진합니다.
*   **G7:** 히로시마 AI 프로세스를 통해 AI 기업 대상의 국제 행동강령에 합의했습니다. 이 강령은 AI 수명주기 전반에 걸친 위험 평가와 완화, 투명성과 책임성 보장, 정보 공유, 보안 통제, 콘텐츠 인증 및 출처 확인 등의 조치를 요구하며 기업의 자발적인 채택을 권고합니다.
*   **영국 AI 안전성 정상회의:** 28개국이 참가하여 AI 위험에 공동 대응하는 '블레츨리 선언'을 발표했습니다. 이 선언은 AI 안전 보장을 위한 국가, 국제기구, 기업, 시민사회, 학계 등 모든 이해관계자의 협력을 강조하며, 특히 최첨단 AI 시스템 개발 기업의 안전 평가 및 적절한 조치 의무를 지적합니다. 또한 영국은 정부 주도의 첨단 AI 시스템 안전 테스트를 주도할 'AI 안전 연구소' 설립을 발표했습니다.
*   **미국 법원:** 생성 AI 기업에 제기된 예술가들의 저작권 소송을 기각했습니다.
*   **미국 연방거래위원회:** 저작권청에 소비자 보호 및 경쟁 측면의 AI 의견서를 제출했습니다.
*   **EU AI 법:** 3자 협상이 기반 모델 규제 관련 견해차로 난항을 겪고 있습니다.

**2. 기업/산업 동향:**
*   **미국 프런티어 모델 포럼:

In [21]:
response = query(client, model_id, file_store_name, contents="삼성전자가 개발한 AI의 이름은 무엇인가요?")
print(response.text)

삼성전자가 개발한 AI의 이름은 '삼성 가우스'입니다. 삼성 가우스는 2023년 11월 8일 '삼성 AI 포럼 2023' 행사에서 처음 공개되었습니다. 이 AI 모델은 정규분포 이론을 정립한 수학자 가우스의 이름을 본떠 만들어졌으며, 다양한 상황에 최적화된 크기의 모델을 선택할 수 있습니다.

삼성 가우스는 세 가지 모델로 구성되어 있습니다:
*   **언어 모델**: 텍스트를 생성하며, 메일 작성, 문서 요약, 번역 등의 업무를 지원합니다. 클라우드 및 온디바이스를 대상으로 다양한 모델로 구성됩니다.
*   **코드 모델**: 코드를 생성하며, AI 코딩 어시스턴트 '코드아이(code.i)'를 통해 대화형 인터페이스로 사내 소프트웨어 개발에 최적화된 서비스를 제공합니다.
*   **이미지 모델**: 창의적인 이미지를 생성하거나 기존 이미지를 원하는 대로 변경하고, 저해상도 이미지를 고해상도로 전환하는 기능을 지원합니다.

삼성 가우스는 온디바이스에서 작동하도록 설계되어 사용자의 정보가 외부로 유출될 위험이 없다는 장점을 가지고 있습니다. 삼성전자는 이 생성형 AI 모델을 다양한 제품에 단계적으로 탑재할 계획입니다.


#### **4. 답변에 활용된 청크 내용 확인**

In [24]:
import textwrap

grounding = response.candidates[0].grounding_metadata

if grounding and grounding.grounding_chunks:
    print(f"Found {len(grounding.grounding_chunks)} grounding chunks.")
    for i, chunk in enumerate(grounding.grounding_chunks, start=1):
        print(f"\nChunk {i} source: {chunk.retrieved_context.title}")
        print("Chunk text:")
        print(textwrap.indent(chunk.retrieved_context.text[:150] + "...", "  "))
else:
    print("No grounding metadata found.")

Found 5 grounding chunks.

Chunk 1 source: SPRI_AI_Brief_2023_12
Chunk text:
  가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유

  ● 언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스, 온디바이스 작동 지원

  ■ 삼성전자가 2023년 11월 8일 열린 '삼성 AI 포럼 2023' 행사에서 자체 개발한 생성 AI 모델
   '삼성 ...

Chunk 2 source: SPRI_AI_Brief_2023_12
Chunk text:
  알리바바 클라우드는 여러 산업 영역에서 생성 AI를 활용해 사업 성과를 개선할 수 있도록 지원
   하는 산업별 모델도 출시

  • 산업 영역은 고객지원, 법률 상담, 의료, 금융, 문서관리, 오디오와 동영상 관리, 코드 개발, 캐릭터
   제작을 포함

  ■ 알리바바 클라우드는...

Chunk 3 source: SPRI_AI_Brief_2023_12
Chunk text:
  픽셀(Pixel)과 경쟁할 것으로 예상

  출처: 삼성전자, '삼성 AI 포럼'서 자체 개발 생성형 AI '삼성 가우스' 공개, 2023.11.08.

  삼성전자, '삼성 개발자 콘퍼런스 코리아 2023' 개최, 2023.11.14.

  TechRepublic, Samsun...

Chunk 4 source: SPRI_AI_Brief_2023_12
Chunk text:
  --- PAGE 1 ---

  SPRI AI Brief

  인공지능 산업의 최신 동향

  2023년 12월호

  SPRI 소프트웨어정책연구소
   Software Policy & Research Institute


  --- PAGE 2 ---

  CONTENTS

  Ⅰ. 인공지능 산...

Chunk 5 source: SPRI_AI_Brief_2023_12
Chunk text:
  개발을 위한 새로운 접근 방식 연구를 수행
   * 편향된 훈련 데이터에 대한 분석기술, 민감한 정보를

In [38]:
from IPython.display import Markdown, display

# Accumulate the response as it is annotated.
annotated_response_parts = []

if not grounding or not grounding.grounding_supports:
    print("No grounding metadata or supports found for annotation.")
else:
    # UTF-8 바이트로 인코딩
    response_bytes = response.text.encode('utf-8')

    cursor = 0
    for support in grounding.grounding_supports:
        start_index = support.segment.start_index or 0

        if start_index > cursor:
            # 바이트 슬라이싱 후 디코딩
            annotated_response_parts.append(
                response_bytes[cursor:start_index].decode('utf-8')
            )

        # Construct the superscript citation from chunk IDs
        chunk_ids = ', '.join(map(str, support.grounding_chunk_indices))
        citation = f"[{chunk_ids}]"

        # Append the formatted, cited, supported text
        annotated_response_parts.append(f"**{support.segment.text}**{citation}")

        cursor = support.segment.end_index

    # Append any remaining text after the last support
    annotated_response_parts.append(response_bytes[cursor:].decode('utf-8'))

    final_annotated_response = "".join(annotated_response_parts)
    display(Markdown(final_annotated_response))

**삼성전자가 개발한 AI의 이름은 '삼성 가우스'입니다.**[0, 1] **삼성 가우스는 2023년 11월 8일 '삼성 AI 포럼 2023' 행사에서 처음 공개되었습니다.**[0] **이 AI 모델은 정규분포 이론을 정립한 수학자 가우스의 이름을 본떠 만들어졌으며, 다양한 상황에 최적화된 크기의 모델을 선택할 수 있습니다.**[0]

삼성 가우스는 세 가지 모델로 구성되어 있습니다:
*   **언어 모델**: 텍스트를 생성하며, 메일 작성, 문서 요약, 번역 등의 업무를 지원합니다. **클라우드 및 온디바이스를 대상으로 다양한 모델로 구성됩니다.**[0]
*   **코드 모델**: 코드를 생성하며, AI 코딩 어시스턴트 '코드아이(code.**i)'를 통해 대화형 인터페이스로 사내 소프트웨어 개발에 최적화된 서비스를 제공합니다.**[0]
***   **이미지 모델**: 창의적인 이미지를 생성하거나 기존 이미지를 원하는 대로 변경하고, 저해상도 이미지를 고해상도로 전환하는 기능을 지원합니다.**[0]

**삼성 가우스는 온디바이스에서 작동하도록 설계되어 사용자의 정보가 외부로 유출될 위험이 없다는 장점을 가지고 있습니다.**[0, 1] **삼성전자는 이 생성형 AI 모델을 다양한 제품에 단계적으로 탑재할 계획입니다.**[0]

#### **5. Document 관리**

In [39]:
# List documents

print(f"Documents in {file_search_store.name}:")

for doc in client.file_search_stores.documents.list(parent=file_search_store.name):
    print(f"- {doc.display_name} ({doc.name})")

Documents in fileSearchStores/test-es7tw2kwt3b8:
- SPRI_AI_Brief_2023_12 (fileSearchStores/test-es7tw2kwt3b8/documents/spriaibrief202312-xq9whafvvfwm)


In [55]:
# Get a document by ID
doc_id = doc.name  # Or set a specific ID here.
sample_doc = client.file_search_stores.documents.get(name=doc_id)
if sample_doc:
    # sample_doc
    print(f"Document details for {sample_doc.display_name}:")
    print(f"\tName: {sample_doc.name}")
    print(f"\tDisplay Name: {sample_doc.display_name}")
    print(f"\tCustom Metadata: {sample_doc.custom_metadata}")
    print(f"\tCreate Time: {sample_doc.create_time}")
    print(f"\tUpdate Time: {sample_doc.update_time}")
    print(f"\tFile Type: {sample_doc.mime_type}")
    print(f"\tFile Size: {sample_doc.size_bytes}")
    print(f"\tState: {sample_doc.state}")

Document details for SPRI_AI_Brief_2023_12:
	Name: fileSearchStores/test-es7tw2kwt3b8/documents/spriaibrief202312-xq9whafvvfwm
	Display Name: SPRI_AI_Brief_2023_12
	Custom Metadata: None
	Create Time: 2025-12-26 03:49:03.773385+00:00
	Update Time: 2025-12-26 03:49:08.215283+00:00
	File Type: application/pdf
	File Size: 975735
	State: DocumentState.STATE_ACTIVE


In [ ]:
# Delete a specific document.
doc_to_be_deleted = doc_id

client.file_search_stores.documents.delete(
    name=doc_to_be_deleted,
    config=types.DeleteDocumentConfig(
        # Set force to delete a non-empty document.
        force=True
    )
)
print(f"Deleted document: {doc_to_be_deleted}")

# Verify deletion
print("\nRemaining documents:")
for doc in client.file_search_stores.documents.list(parent=file_search_store.name):
    print(f"- {doc.display_name}")

#### **5. File Search Stores 관리**

In [56]:
# List
print("Stores:")

for store in client.file_search_stores.list():
    print(f"- {store.name} ({store.display_name})")

Stores:
- fileSearchStores/my-file-search-store-nur2ttwx67bp (My File Search Store)
- fileSearchStores/my-file-search-storesionic-q5v1cb6cbhrj (My File Search Store-Sionic)
- fileSearchStores/test-es7tw2kwt3b8 (test)


In [57]:
# Cleanup
client.file_search_stores.delete(
    name="fileSearchStores/my-file-search-storesionic-q5v1cb6cbhrj", 
    config=types.DeleteFileSearchStoreConfig(force=True)
    )
print(f"Deleted store: {file_search_store.name}")

print()
print("Stores:")
for store in client.file_search_stores.list():
    print(f"- {store.name} ({store.display_name})")

Deleted store: fileSearchStores/test-es7tw2kwt3b8

Stores:
- fileSearchStores/my-file-search-store-nur2ttwx67bp (My File Search Store)
- fileSearchStores/test-es7tw2kwt3b8 (test)
